In [22]:
#In this first step we import the packages that we will use in order to solve the problem
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pulp

# CEE 201: Linear Programming with Jupyter Notebooks

## Example: COVID-19 Ventilators Transportation Problem

### Problem Statement

To promptly respond to COVID-19, two factories distribute ventilators to five hospitals. The city would like to have an interactive computer program which they can run week by week to tell them which factories should supply which hospitals so as to minimize the costs of the whole operation. For example, suppose that at the start of a given week, 1000 ventilators were produced at factory A, and 4000 at factory B, and that the hospitals require 500, 900, 1800, 200, and 700 ventilators respectively. Which factory should supply which hospital?

In [31]:
# Creates a list of all the ventilator supply nodes (ventilator factories)
Factories = [1,2]

# Creates a dictionary for the number of units of supply for each supply node
supply = {1: 1000,
        2: 4000}

# Creates a list of all demand nodes
Hospitals = [1, 2, 3, 4, 5]

# Creates a dictionary for the number of ventilator units of demand for each hospital node
demand = {1: 500,
        2: 900,
        3: 1800,
        4: 200,
        5: 700}

In [39]:
# Creates a list of costs of each transportation path
costs = {#Hospitals
         #1 2 3 4 5
         1:{1:2,2:4,3:5,4:2,5:1},#A  Factory
         2:{1:3,2:1,3:3,4:2,5:3} #B   Factory
}

Below, a list of tuples is created containing all the transportation links and a dictionary called route_var is created which contains the LP variables. The reference keys to the dictionary are the factory name, then the hospital name([“A”][“2”]) , and the data is Route_Tuple. (e.g. [“A”][“2”]: Route_A_2). The lower limit of zero is set, the upper limit of None is set, and the variables are defined to be Integers.

In [40]:
#Set problem variable
# Creates the prob variable to contain the problem data
prob = pulp.LpProblem("Ventilators Distribution Problem",pulp.LpMinimize)
# Creates a list of tuples containing all the possible routes for transport
Routes = [(f,h) for f in Factories for h in Hospitals]
# A dictionary called route_vars is created to contain the referenced variables (the routes)
route_vars = pulp.LpVariable.dicts("Route",(Factories,Hospitals),0,None,pulp.LpInteger)


The objective function is added to the variable prob using a list comprehension. Since route_vars and costs are now dictionaries (with further internal dictionaries), they can be used as if they were tables, as for (f,h) in Routes will cycle through all the combinations/links. Note that i and j could have been used, but f and h are more meaningful.

In [41]:
# The objective function is added to prob first
prob += pulp.lpSum([route_vars[f][h]*costs[f][h] for f in Factories for h in Hospitals])

In [42]:
# The supply maximum constraints are added to prob for each supply node (ventilators factory)
for f in Factories:
    prob += pulp.lpSum([route_vars[f][h] for h in Hospitals]) <= supply[f]
# The demand minimum constraints are added to prob for each demand node (hospital)
for h in Hospitals:
    prob += pulp.lpSum([route_vars[f][h] for f in Factories]) >= demand[h]

In [44]:
# Solve our problem
prob.solve()
pulp.LpStatus[prob.status]

'Optimal'

In [45]:
total_cost = pulp.value(prob.objective)

print("The total cost in $ is",total_cost)

The total cost in $ is 8600.0


In [48]:
for v in prob.variables():
    print(v.name,"=",v.varValue)

Route_1_1 = 300.0
Route_1_2 = 0.0
Route_1_3 = 0.0
Route_1_4 = 0.0
Route_1_5 = 700.0
Route_2_1 = 200.0
Route_2_2 = 900.0
Route_2_3 = 1800.0
Route_2_4 = 200.0
Route_2_5 = 0.0
